Code that goes along with the Airflow tutorial located at:
https://github.com/airbnb/airflow/blob/master/airflow/example_dags/tutorial.py

In [1]:
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from datetime import datetime, timedelta


default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2015, 6, 1),
    'email': ['airflow@airflow.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
}

dag = DAG('tutorial', default_args=default_args)

# t1, t2 and t3 are examples of tasks created by instantiating operators
t1 = BashOperator(
    task_id='print_date',
    bash_command='date',
    dag=dag)

t2 = BashOperator(
    task_id='sleep',
    bash_command='sleep 5',
    retries=3,
    dag=dag)

templated_command = """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7)}}"
        echo "{{ params.my_param }}"
    {% endfor %}
"""

t3 = BashOperator(
    task_id='templated',
    bash_command=templated_command,
    params={'my_param': 'Parameter I passed in'},
    dag=dag)

t2.set_upstream(t1)
t3.set_upstream(t1)

[2017-09-18 09:59:12,696] {__init__.py:57} INFO - Using executor SequentialExecutor


In [4]:
# print the list of active DAGs

!/Users/superizer/anaconda/envs/tensorflow/bin/airflow list_dags

[2017-09-18 10:07:20,223] {__init__.py:57} INFO - Using executor SequentialExecutor
[2017-09-18 10:07:20,777] {models.py:167} INFO - Filling up the DagBag from /Users/superizer/airflow/dags


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
example_bash_operator
example_branch_dop_operator_v3
example_branch_operator
example_http_operator
example_passing_params_via_test_command
example_python_operator
example_short_circuit_operator
example_skip_dag
example_subdag_operator
example_subdag_operator.section-1
example_subdag_operator.section-2
example_trigger_controller_dag
example_trigger_target_dag
example_xcom
latest_only
latest_only_with_trigger
test_utils
tutorial



In [5]:
# prints the list of tasks the "tutorial" dag_id

!/Users/superizer/anaconda/envs/tensorflow/bin/airflow list_tasks tutorial

[2017-09-18 10:09:58,631] {__init__.py:57} INFO - Using executor SequentialExecutor
[2017-09-18 10:09:59,183] {models.py:167} INFO - Filling up the DagBag from /Users/superizer/airflow/dags
print_date
sleep
templated


In [6]:
# prints the hierarchy of tasks in the tutorial DAG

!/Users/superizer/anaconda/envs/tensorflow/bin/airflow list_tasks tutorial --tree

[2017-09-18 10:10:15,518] {__init__.py:57} INFO - Using executor SequentialExecutor
[2017-09-18 10:10:16,054] {models.py:167} INFO - Filling up the DagBag from /Users/superizer/airflow/dags
<Task(BashOperator): sleep>
    <Task(BashOperator): print_date>
<Task(BashOperator): templated>
    <Task(BashOperator): print_date>


In [10]:
# command layout: command subcommand dag_id task_id date

# testing print_date
!/Users/superizer/anaconda/envs/tensorflow/bin/airflow test tutorial print_date 2017-09-18

[2017-09-18 11:19:22,920] {__init__.py:57} INFO - Using executor SequentialExecutor
[2017-09-18 11:19:23,483] {models.py:167} INFO - Filling up the DagBag from /Users/superizer/airflow/dags
Traceback (most recent call last):
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/sqlalchemy/engine/base.py", line 1182, in _execute_context
    context)
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: task_instance

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/superizer/anaconda/envs/tensorflow/bin/airflow", line 28, in <module>
    args.func(args)
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/airflow/bin/cli.py", line 585, in test
    ti.run(ignore_task_deps=True, ignore_ti_state=True, test_mode=T

In [9]:
!date 2015-06-01

date: illegal time format
usage: date [-jnRu] [-d dst] [-r seconds] [-t west] [-v[+|-]val[ymwdHMS]] ... 
            [-f fmt date | [[[mm]dd]HH]MM[[cc]yy][.ss]] [+format]


In [11]:
# testing sleep
!/Users/superizer/anaconda/envs/tensorflow/bin/airflow test tutorial sleep 2015-06-01

[2017-09-18 11:22:19,085] {__init__.py:57} INFO - Using executor SequentialExecutor
[2017-09-18 11:22:19,660] {models.py:167} INFO - Filling up the DagBag from /Users/superizer/airflow/dags
Traceback (most recent call last):
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/sqlalchemy/engine/base.py", line 1182, in _execute_context
    context)
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: task_instance

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/superizer/anaconda/envs/tensorflow/bin/airflow", line 28, in <module>
    args.func(args)
  File "/Users/superizer/anaconda/envs/tensorflow/lib/python3.6/site-packages/airflow/bin/cli.py", line 585, in test
    ti.run(ignore_task_deps=True, ignore_ti_state=True, test_mode=T